In [1]:
import json
import yaml
from json_schema_to_glue_columns import *

In [2]:
def load_schema(schema_file_location):
    with open(schema_file_location) as schema_file:
        schema = json.load(schema_file)
    return schema

In [3]:
def load_config(config_file_location):
    with open(config_file_location) as config_file:
        config = yaml.safe_load(config_file)
    return config

In [4]:
def create_partition_keys(partition_key_list):
    partition_keys = []
    for partition_key in partition_key_list:
        key_name, key_type = partition_key.split(":")
        partition_keys.append({
            "Name": key_name.strip(),
            "Type": key_type.strip()
        })
    return partition_keys

In [5]:
partition_key_list = ["year:int", "month:int", "day:int", "hour:int", "minute:int"]
partition_keys = create_partition_keys(partition_key_list)
partition_keys

[{'Name': 'year', 'Type': 'int'},
 {'Name': 'month', 'Type': 'int'},
 {'Name': 'day', 'Type': 'int'},
 {'Name': 'hour', 'Type': 'int'},
 {'Name': 'minute', 'Type': 'int'}]

In [6]:
schema_file_location_1 = "./sample1.schema.json"
config_file_location_1 = "./sample1.config.yml"

schema_1 = load_schema(schema_file_location_1)
config_1 = load_config(config_file_location_1)

In [7]:
s3_location_raw = f"{config_1['s3_location']}/raw"
s3_location_staging = f"{config_1['s3_location']}/staging"
pipeline_type = config_1["pipeline_type"]

In [8]:
glue_columns_raw = convert_json_schema_to_glue_columns(schema_1)
glue_columns_raw

[{'Name': 'id', 'Type': 'DOUBLE'},
 {'Name': 'isbn', 'Type': 'STRING'},
 {'Name': 'author', 'Type': 'STRUCT<lastname:STRING,firstname:STRING>'},
 {'Name': 'editor', 'Type': 'STRUCT<lastname:STRING,firstname:STRING>'},
 {'Name': 'title', 'Type': 'STRING'},
 {'Name': 'category', 'Type': 'ARRAY<STRING>'},
 {'Name': 'tags', 'Type': 'ARRAY<STRUCT<key:STRING,value:STRING>>'}]

### Sample Glue create_table API call

```
import boto3

client = boto3.client('glue')

response = client.create_table(
    DatabaseName="raw",
    TableInput={
        "Name": "sample1",
        "TableType": "EXTERNAL_TABLE",
        "Parameters": {
            "classification": "json",
            "typeOfData": "file"
        },
        "StorageDescriptor": {
            "Columns": [
                {
                    "Name": "id",
                    "Type": "DOUBLE"
                },
                {
                    "Name": "isbn",
                    "Type": "STRING"
                },
                {
                    "Name": "author",
                    "Type": "STRUCT<lastname:STRING,firstname:STRING>"
                },
                {
                    "Name": "editor",
                    "Type": "STRUCT<lastname:STRING,firstname:STRING>"
                },
                {
                    "Name": "title",
                    "Type": "STRING"
                },
                {
                    "Name": "category",
                    "Type": "ARRAY<STRING>"
                },
                {
                    "Name": "tags",
                    "Type": "ARRAY<STRUCT<key:STRING,value:STRING>>"
                }
            ],
            "Location": "s3://bucket/prefix/raw",
            "InputFormat": "org.apache.hadoop.mapred.TextInputFormat",
            "OutputFormat": "org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat",
            "SerdeInfo": {
                "SerializationLibrary": "org.openx.data.jsonserde.JsonSerDe",
                "Parameters": {
                    "serialization.format": "1"
                }
            }
        },
        "PartitionKeys": [
            {
                "Name": "year",
                "Type": "int"
            },
            {
                "Name": "month",
                "Type": "int"
            },
            {
                "Name": "day",
                "Type": "int"
            },
            {
                "Name": "hour",
                "Type": "int"
            },
            {
                "Name": "minute",
                "Type": "int"
            }
        ]
    }
)
```

In [9]:
glue_columns_staging = convert_json_schema_to_glue_columns(schema_1, flatten=True, delimiter='__')
glue_columns_staging

[{'Name': 'id', 'Type': 'DOUBLE'},
 {'Name': 'isbn', 'Type': 'STRING'},
 {'Name': 'author__lastname', 'Type': 'STRING'},
 {'Name': 'author__firstname', 'Type': 'STRING'},
 {'Name': 'editor__lastname', 'Type': 'STRING'},
 {'Name': 'editor__firstname', 'Type': 'STRING'},
 {'Name': 'title', 'Type': 'STRING'},
 {'Name': 'category', 'Type': 'ARRAY<STRING>'},
 {'Name': 'tags', 'Type': 'ARRAY<STRUCT<key:STRING,value:STRING>>'}]

In [10]:
if pipeline_type.lower() == "scd1":
    glue_columns_staging.append({
        "Name": "last_updated",
        "Type": "STRING"
    })
elif pipeline_type.lower() == "scd2":
    glue_columns_staging.append({
        "Name": "last_updated",
        "Type": "STRING"
    },
    {
        "Name": "active",
        "Type": "STRING"
    })

glue_columns_staging

[{'Name': 'id', 'Type': 'DOUBLE'},
 {'Name': 'isbn', 'Type': 'STRING'},
 {'Name': 'author__lastname', 'Type': 'STRING'},
 {'Name': 'author__firstname', 'Type': 'STRING'},
 {'Name': 'editor__lastname', 'Type': 'STRING'},
 {'Name': 'editor__firstname', 'Type': 'STRING'},
 {'Name': 'title', 'Type': 'STRING'},
 {'Name': 'category', 'Type': 'ARRAY<STRING>'},
 {'Name': 'tags', 'Type': 'ARRAY<STRUCT<key:STRING,value:STRING>>'},
 {'Name': 'last_updated', 'Type': 'STRING'}]

### Sample Glue create_table API call

```
import boto3

client = boto3.client('glue')

response = client.create_table(
    DatabaseName="staging",
    "TableInput": {
        "Name": "sample1",
        "TableType": "EXTERNAL_TABLE",
        "Parameters": {
            "classification": "parquet",
            "typeOfData": "file"
        },
        "StorageDescriptor": {
            "Columns": [
                {
                    "Name": "id",
                    "Type": "DOUBLE"
                },
                {
                    "Name": "isbn",
                    "Type": "STRING"
                },
                {
                    "Name": "author__lastname",
                    "Type": "STRING"
                },
                {
                    "Name": "author__firstname",
                    "Type": "STRING"
                },
                {
                    "Name": "editor__lastname",
                    "Type": "STRING"
                },
                {
                    "Name": "editor__firstname",
                    "Type": "STRING"
                },
                {
                    "Name": "title",
                    "Type": "STRING"
                },
                {
                    "Name": "category",
                    "Type": "ARRAY<STRING>"
                },
                {
                    "Name": "tags",
                    "Type": "ARRAY<STRUCT<key:STRING,value:STRING>>"
                },
                {
                    "Name": "last_updated",
                    "Type": "STRING"
                }
            ],
            "Location": "s3://bucket/prefix/staging",
            "InputFormat": "org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat",
            "OutputFormat": "org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat",
            "SerdeInfo": {
                "SerializationLibrary": "org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe"
            }
        },
        "PartitionKeys": [
            {
                "Name": "year",
                "Type": "int"
            },
            {
                "Name": "month",
                "Type": "int"
            },
            {
                "Name": "day",
                "Type": "int"
            },
            {
                "Name": "hour",
                "Type": "int"
            },
            {
                "Name": "minute",
                "Type": "int"
            }
        ]
    }
)
```